In [ ]:
import numpy as np
from pathlib import Path
import re

import matplotlib.pyplot as plt
import seaborn as sns

from data_preparation.preprocessing.cleaner import Cleaner
from data_preparation.preprocessing.outlier_handler import OutlierHandler
from data_preparation.preprocessing.scaler import Scaler

In [ ]:
DATA_DIR = Path("../data")
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

In [ ]:
file = open(RAW_DATA_DIR / "creditcard.csv", "r", encoding="utf-8")

cols = {}
pattern = re.compile(r"""((?:[^,"']|"[^"]*"|'[^']*')+)""")

first_line = file.readline()
entries = pattern.findall(first_line.strip())

for entry in entries:
    cols[entry] = []

for line in file:
    entries = pattern.findall(line.strip())

    for i, entry in enumerate(entries):
        key = list(cols.keys())[i]
        cols[key].append(entry)

# 1. Làm sạch dữ liệu thô

## 1.1 Làm sạch tên cột

In [ ]:
def clean_column_name(cols):
    new_cols = {}
    for key in cols.keys():
        new_key = Cleaner.clean_string(key)
        new_cols[new_key] = cols[key]
    return new_cols


cols = clean_column_name(cols)

## 1.2 Chuyển tất cả các cột thành kiểu dữ liệu số

In [ ]:
cols = Cleaner.convert_cols_to_numeric(cols)
cols

# 2. Chuẩn hoá dữ liệu (Normalization)

In [ ]:
# dựa trên kết quả eda

cols_with_outliers = OutlierHandler.get_cols_with_outliers(
    cols,
    threshold_percentage=10,
)
cols_with_outliers

## 2.1 Xử lí các giá trị ngoại lai

In [ ]:
for col in cols_with_outliers:
    print(f"Clipping outliers in column: {col}")

    print(f"Before clipping outliers: {OutlierHandler.count_outliers(cols[col])}")

    cols[col] = OutlierHandler.clip_outliers(
        col=cols[col],
        lower_percentile=1,
        upper_percentile=99,
    )

    print(f"After clipping outliers: {OutlierHandler.count_outliers(cols[col])}")
    print()

for col in cols_with_outliers:
    print(f"Clipping outliers in column: {col}")

    print(f"Before clipping outliers: {OutlierHandler.count_outliers(cols[col])}")

    # Clip theo IQR bounds
    cols[col] = OutlierHandler.clip_outliers_by_iqr(cols[col])

    print(f"After clipping outliers: {OutlierHandler.count_outliers(cols[col])}")
    print()

## 2.2 Scale dữ liệu

In [ ]:
cols_to_robust_scale = [
    "time",
    "amount",
]

for robust_scale_col in cols_to_robust_scale:
    print(f"Robust scaling column: {robust_scale_col}")
    cols[robust_scale_col] = Scaler.robust_scale_column(cols[robust_scale_col])
    print()

### Lưu ý:
Trước khi chạy PCA transformation thì cột đó phải được scale vì vậy ta không cần phải scale các cột này như đã nói trong lúc eda

In [ ]:
%%script true

cols_to_standard_scale = list(set(cols.keys()) - set(cols_to_robust_scale) - {"class"})
cols_to_standard_scale

for standard_scale_col in cols_to_standard_scale:
    print(f"Standard scaling column: {standard_scale_col}")
    cols[standard_scale_col] = Scaler.standardize_column(cols[standard_scale_col])
    print()

In [ ]:
nums_rows_to_display = 5

for col in cols:
    print(f"First {nums_rows_to_display} entries in column '{col}':")
    print(cols[col][:nums_rows_to_display])
    print()

# 3. Lưu dữ liệu đã được xử lí

In [ ]:
file = open(PROCESSED_DATA_DIR / "creditcard_processed.csv", "w", encoding="utf-8")
header = ",".join(cols.keys())
file.write(header + "\n")

for i in range(len(cols["class"])):
    row_entries = []
    for col in cols:
        row_entries.append(str(cols[col][i]))
    row = ",".join(row_entries)
    file.write(row + "\n")

file.close()